<a href="https://colab.research.google.com/github/rajlm10/D2L-Torch/blob/main/D2L_Seq_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install d2l
#Restart runtime

# Utils 

In [33]:
import collections
import re
from d2l import torch as d2l
import torch
import math
import random
from torch import nn
from torch.nn import functional as F 


In [34]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt', '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():
  """Load the time machine dataset into a list of text lines.""" 
  with open(d2l.download('time_machine'), 'r') as f:
    lines = f.readlines()
  return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine() 
print(f'# text lines: {len(lines)}') 
print(lines[0])
print(lines[10])

# text lines: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [35]:
#Tokenization
def tokenize(lines,token='word'):
  if token=='word':
    return [line.split() for line in lines]
  if token=='char':
    return [list(line) for line in lines]
  else:
    print('ERROR: unknown token type: ' + token)

In [36]:
class Vocab:
  def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
    if tokens is None:
      tokens = []
    if reserved_tokens is None: 
      reserved_tokens = []
    # Sort according to frequencies
    counter = count_corpus(tokens)
    self._token_freqs = sorted(counter.items(), key=lambda x: x[1],reverse=True)

    #Build vocab on init
    self.idx_to_token=['<unk>']+reserved_tokens #List
    self.token_to_idx={token:idx for idx,token in enumerate(self.idx_to_token)} #Dict

    for token,freq in self._token_freqs:
      #Don't include tokens with freq<min_freq in the vocab
      if freq<min_freq:
        break
      if token not in self.idx_to_token:
        self.idx_to_token.append(token)
        self.token_to_idx[token]=len(self.idx_to_token)-1

  def __len__(self):
    return len(self.idx_to_token)

  @property
  def unk(self):
    # Index for the unknown token
    return 0
  
  @property
  def token_freqs(self):
    # Counter object 
    return self._token_freqs
  
  def __getitem__(self,tokens):
    if not isinstance(tokens, (list, tuple)):
      return self.token_to_idx.get(tokens, self.unk) 
    #If tokens is a list 
    return [self.__getitem__(token) for token in tokens]

  def to_tokens(self, indices):
    if not isinstance(indices, (list, tuple)):
      return self.idx_to_token[indices]
    #If indices is a list
    return [self.idx_to_token[index] for index in indices]


def count_corpus(tokens):
  """Count token frequencies."""
  # Here `tokens` is a 1D list or 2D list
  if len(tokens) == 0 or isinstance(tokens[0], list):
    # Flatten a list of token lists into a 1D list
    tokens = [token for line in tokens for token in line]
  #If its already a 1D list,return  
  return collections.Counter(tokens)


In [37]:
def load_corpus_time_machine(max_tokens=-1,level='char'):
  """Return token indices and the vocabulary of the time machine dataset.""" 
  lines = read_time_machine()
  tokens = tokenize(lines, level)
  vocab = Vocab(tokens)
  # Call getitem on a single token and append it to a list
  corpus = [vocab[token] for line in tokens for token in line]
  if max_tokens > 0:
    corpus = corpus[:max_tokens] #First max_tokens tokens
  return corpus, vocab


In [38]:
import random
def seq_data_iter_random(corpus, batch_size, num_steps):
  """Generate a minibatch of subsequences using random sampling."""
  # Start with a random offset (inclusive of `num_steps - 1`) to partition a # sequence
  corpus = corpus[random.randint(0, num_steps - 1):]
  # Subtract 1 since we need to account for labels
  num_subseqs = (len(corpus) - 1) // num_steps
  # The starting indices for subsequences of length `num_steps` 
  initial_indices = list(range(0, num_subseqs * num_steps, num_steps))
  # In random sampling, the subsequences from two adjacent random
  # minibatches during iteration are not necessarily adjacent on the
  # original sequence
  random.shuffle(initial_indices)

  def data(pos):
  # Return a sequence of length `num_steps` starting from `pos` 
    return corpus[pos: pos + num_steps]
  
  num_batches = num_subseqs // batch_size
  for i in range(0, batch_size * num_batches, batch_size):
    # Here, `initial_indices` contains randomized starting indices for # subsequences
    initial_indices_per_batch = initial_indices[i: i + batch_size]
    X = [data(j) for j in initial_indices_per_batch]
    Y = [data(j + 1) for j in initial_indices_per_batch] 
    yield torch.tensor(X), torch.tensor(Y)

In [39]:
def seq_data_iter_sequential(corpus, batch_size, num_steps):
  """Generate a minibatch of subsequences using sequential partitioning.""" 
  # Start with a random offset to partition a sequence
  offset = random.randint(0, num_steps)
  num_tokens = ((len(corpus) - offset - 1) // batch_size) * batch_size
  Xs = torch.tensor(corpus[offset: offset + num_tokens])
  Ys = torch.tensor(corpus[offset + 1: offset + 1 + num_tokens])
  Xs, Ys = Xs.reshape(batch_size, -1), Ys.reshape(batch_size, -1) 
  num_batches = Xs.shape[1] // num_steps
  for i in range(0, num_steps * num_batches, num_steps):
    X = Xs[:, i: i + num_steps]
    Y = Ys[:, i: i + num_steps] 
    yield X, Y

In [40]:
class SeqDataLoader:
  """An iterator to load sequence data."""
  def __init__(self, batch_size, num_steps, use_random_iter, max_tokens):
    if use_random_iter:
      self.data_iter_fn = seq_data_iter_random
    else:
      self.data_iter_fn = seq_data_iter_sequential
    
    self.corpus, self.vocab = load_corpus_time_machine(max_tokens) 
    self.batch_size, self.num_steps = batch_size, num_steps

  def __iter__(self):
    return self.data_iter_fn(self.corpus, self.batch_size, self.num_steps)

In [41]:
def load_data_time_machine(batch_size, num_steps, use_random_iter=False, max_tokens=10000):
  """Return the iterator and the vocabulary of the time machine dataset."""
  data_iter = SeqDataLoader(batch_size, num_steps, use_random_iter, max_tokens)
  return data_iter, data_iter.vocab

# RNNs 

In [42]:
def predict(prefix, num_preds, net, vocab, device): 
  """Generate new characters following the `prefix`."""
  state = net.begin_state(batch_size=1, device=device) 
  outputs = [vocab[prefix[0]]] # gives [token]
  get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1)) 
  for y in prefix[1:]: # Warm-up period
    _, state = net(get_input(), state)
    outputs.append(vocab[y]) #Append input text i.e is prefix

  for _ in range(num_preds): # Predict `num_preds` steps
    y, state = net(get_input(), state) #Keep updating state and input
    outputs.append(int(y.argmax(dim=1).reshape(1))) 
  return ''.join([vocab.idx_to_token[i] for i in outputs])

In [43]:
def grad_clipping(net, theta):
  """Clip the gradient."""
  if isinstance(net, nn.Module):
    params = [p for p in net.parameters() if p.requires_grad] 
  else:
    params = net.params
  norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params)) 
  if norm > theta:
    for param in params: 
      param.grad[:] *= theta / norm

In [44]:
def train_epoch(net, train_iter, loss, optimizer, device, use_random_iter): 
  """Train a net within one epoch"""
  state, timer = None, d2l.Timer()
  metric = d2l.Accumulator(2) # Sum of training loss, no. of tokens
  for X, Y in train_iter:
    if state is None or use_random_iter:
      # Initialize `state` when either it is the first iteration or 
      # using random sampling
      state = net.begin_state(batch_size=X.shape[0], device=device)
    else:
      if isinstance(net, nn.Module) and not isinstance(state, tuple):
        # `state` is a tensor for `nn.GRU`
        state.detach_() 
      else:
        # `state` is a tuple of tensors for `nn.LSTM` and # for our custom scratch implementation
        for s in state:
          s.detach_() 
    y = Y.T.reshape(-1)
    X, y = X.to(device), y.to(device)
    y_hat, state = net(X, state)
    l = loss(y_hat, y.long()).mean()
    
    if isinstance(optimizer, torch.optim.Optimizer):
      optimizer.zero_grad() 
      l.backward() 
      grad_clipping(net, 1) 
      optimizer.step()

    metric.add(l * y.numel(), y.numel())
  return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()

In [45]:
def train(net, train_iter, vocab,loss, optimizer, num_epochs, device, use_random_iter=False):
  """Train a model"""
  for epoch in range(num_epochs):
    ppl, speed = train_epoch(net, train_iter, loss, optimizer, device, use_random_iter)
    if (epoch + 1) % 50 == 0:
      print(f'perplexity {ppl:.3f}, {speed:.1f} tokens/sec on {str(device)}') 

In [46]:
batch_size, num_steps = 32, 35
train_iter, vocab = load_data_time_machine(batch_size, num_steps)

In [47]:
class RNNModel(nn.Module):
  """The RNN model."""
  def __init__(self, rnn_layer, vocab_size, **kwargs):
    super(RNNModel, self).__init__(**kwargs)
    self.rnn = rnn_layer #Can be a RNN,GRU or LSTM
    self.vocab_size = vocab_size
    self.num_hiddens = self.rnn.hidden_size

    # If the RNN is bidirectional (to be introduced later), 
    # `num_directions` should be 2, else it should be 1.
    if not self.rnn.bidirectional:
      self.num_directions = 1
      self.linear = nn.Linear(self.num_hiddens, self.vocab_size)
    else:
      self.num_directions = 2
      self.linear = nn.Linear(self.num_hiddens * 2, self.vocab_size)

  def forward(self, inputs, state):
    X = F.one_hot(inputs.T.long(), self.vocab_size) #inputs.T=num_steps X batch_size
    X = X.to(torch.float32)
    Y, state = self.rnn(X, state)
    # The fully connected layer will first change the shape of `Y` to # (`num_steps` * `batch_size`, `num_hiddens`). 
    #Its output shape after the linear layer is # (`num_steps` * `batch_size`, `vocab_size`).
    output = self.linear(Y.reshape((-1, Y.shape[-1])))
    return output, state

  def begin_state(self, device, batch_size=1): 
    if not isinstance(self.rnn, nn.LSTM):
    # `nn.GRU` takes a tensor as hidden state
      return torch.zeros((self.num_directions * self.rnn.num_layers, batch_size, self.num_hiddens),device=device)
    
    else:
    # `nn.LSTM` takes a tuple of hidden states return for C and H
      ( torch.zeros((self.num_directions * self.rnn.num_layers, batch_size, self.num_hiddens), device=device),
       torch.zeros((self.num_directions * self.rnn.num_layers, batch_size, self.num_hiddens), device=device) )


In [48]:
device = d2l.try_gpu()
num_hiddens=256
net = RNNModel(nn.RNN(len(vocab), num_hiddens), vocab_size=len(vocab))
net = net.to(device)
predict('time traveller', 10, net, vocab, device)

'time traveller          '

In [49]:
num_epochs, lr = 500, 1

loss=nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr)
train(net, train_iter, vocab,loss,optimizer, num_epochs, device)

perplexity 7.334, 114887.5 tokens/sec on cuda:0
perplexity 3.690, 110155.8 tokens/sec on cuda:0
perplexity 2.029, 109621.7 tokens/sec on cuda:0
perplexity 1.591, 109958.8 tokens/sec on cuda:0
perplexity 1.478, 111110.3 tokens/sec on cuda:0
perplexity 1.357, 110368.3 tokens/sec on cuda:0
perplexity 1.337, 109984.6 tokens/sec on cuda:0
perplexity 1.341, 108889.6 tokens/sec on cuda:0
perplexity 1.329, 110086.7 tokens/sec on cuda:0
perplexity 1.254, 109893.9 tokens/sec on cuda:0


In [51]:
predict('time traveller', 50, net, vocab, device)

'time traveller froce ted ale predery aid hey timickon time erd a'

# GRU

In [56]:
gru_layer = nn.GRU(len(vocab), num_hiddens)
model = RNNModel(gru_layer, len(vocab))
model = model.to(device)
predict('time traveller', 50, model, vocab, device)


'time travellerzzlzzlzzzlzzzlzzzlzzzlzzzlzzzlzzzlzzzlzzzlzzzlzzzl'

In [57]:
loss=nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr)
train(model, train_iter, vocab,loss,optimizer, num_epochs, device)

perplexity 9.561, 122190.3 tokens/sec on cuda:0
perplexity 7.706, 124487.7 tokens/sec on cuda:0
perplexity 5.952, 123327.1 tokens/sec on cuda:0
perplexity 3.616, 122662.1 tokens/sec on cuda:0
perplexity 1.648, 121400.1 tokens/sec on cuda:0
perplexity 1.154, 116743.0 tokens/sec on cuda:0
perplexity 1.074, 122402.0 tokens/sec on cuda:0
perplexity 1.050, 124860.3 tokens/sec on cuda:0
perplexity 1.048, 102389.3 tokens/sec on cuda:0
perplexity 1.051, 120393.5 tokens/sec on cuda:0


In [58]:
predict('time traveller', 50, model, vocab, device)


'time travelleryou can show black is white by argument said filby'

# LSTM

In [59]:
lstm_layer = nn.LSTM(len(vocab), num_hiddens)
model = RNNModel(lstm_layer, len(vocab))
model = model.to(device)
predict('time traveller', 50, model, vocab, device)


'time travellerrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr'

In [60]:
loss=nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr)
train(model, train_iter, vocab,loss,optimizer, num_epochs, device)

perplexity 11.062, 104974.2 tokens/sec on cuda:0
perplexity 8.569, 103188.3 tokens/sec on cuda:0
perplexity 6.704, 104276.3 tokens/sec on cuda:0
perplexity 4.381, 102547.4 tokens/sec on cuda:0
perplexity 2.291, 103186.3 tokens/sec on cuda:0
perplexity 1.303, 104121.7 tokens/sec on cuda:0
perplexity 1.091, 103596.5 tokens/sec on cuda:0
perplexity 1.052, 102812.0 tokens/sec on cuda:0
perplexity 1.062, 102441.2 tokens/sec on cuda:0
perplexity 1.045, 103102.5 tokens/sec on cuda:0


In [61]:
predict('time traveller', 50, model, vocab, device)


'time traveller for so it will be convenient to speak of himwas e'

# Stacked LSTM 

In [64]:
num_stacks=2
lstm_layer = nn.LSTM(len(vocab), num_hiddens,num_stacks)
model = RNNModel(lstm_layer, len(vocab))
model = model.to(device)
predict('time traveller', 50, model, vocab, device)


'time travellertttttttttttttttttttttttttttttttttttttttttttttttttt'

In [65]:
loss=nn.CrossEntropyLoss()
lr=2 #complex architecture
optimizer = torch.optim.SGD(model.parameters(), lr)
train(model, train_iter, vocab,loss,optimizer, num_epochs, device)

perplexity 14.677, 80719.8 tokens/sec on cuda:0
perplexity 9.309, 80687.4 tokens/sec on cuda:0
perplexity 5.698, 81486.1 tokens/sec on cuda:0
perplexity 1.854, 81052.3 tokens/sec on cuda:0
perplexity 1.107, 81331.2 tokens/sec on cuda:0
perplexity 1.048, 81358.7 tokens/sec on cuda:0
perplexity 1.033, 80827.4 tokens/sec on cuda:0
perplexity 1.033, 80866.8 tokens/sec on cuda:0
perplexity 1.031, 82017.4 tokens/sec on cuda:0
perplexity 1.021, 80632.2 tokens/sec on cuda:0


In [66]:
predict('time traveller', 50, model, vocab, device)


'time traveller for so it will be convenient to speak of himwas e'

# Bidirectional LSTM

In [ ]:
#To be added